In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
def trim_string(row):

    row = row.split(maxsplit=first_n_words)
    row = ' '.join(row[:first_n_words])

    return row

In [4]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [5]:
train_test_ratio = 0.90
train_valid_ratio = 0.80
first_n_words=250
# "./Models/Experiments/Claims/20epochs/real_fake_claims.csv"
data_path = "./Models/Experiments/Claims/20epochs"

In [6]:
# Read raw data
df_raw = pd.read_csv(data_path+"/real_fake_claims.csv")

# Trim text and titletext to first_n_words
df_raw['Claims'] = df_raw['Claims'].apply(trim_string)

# Split according to label
df_real = df_raw[df_raw['Label'] == 0]
df_fake = df_raw[df_raw['Label'] == 1]



In [7]:
# we need to preprocess such that the fake claims end at <EOS> to get a single full claim
def trim_claims(row):
    sentences = row.split('<EOS>')
    return sentences[0]

df_fake['Claims'] = df_fake['Claims'].apply(trim_claims)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# example
np.asarray(df_fake["Claims"])[2]

'20. The organic light emitting diode as claimed in claim 16, further comprising a passivating layer disposed between the substrate and the plurality of organic light emitting devices, the passivating layer covering the plurality of organic light emitting devices. '

In [9]:
# Train-test split
df_real_full_train, df_real_test = train_test_split(df_real, train_size = train_test_ratio, random_state = 1)
df_fake_full_train, df_fake_test = train_test_split(df_fake, train_size = train_test_ratio, random_state = 1)

# Train-valid split
df_real_train, df_real_valid = train_test_split(df_real_full_train, train_size = train_valid_ratio, random_state = 1)
df_fake_train, df_fake_valid = train_test_split(df_fake_full_train, train_size = train_valid_ratio, random_state = 1)

# Concatenate splits of different labels
df_train = pd.concat([df_real_train, df_fake_train], ignore_index=True, sort=False)
df_valid = pd.concat([df_real_valid, df_fake_valid], ignore_index=True, sort=False)
df_test = pd.concat([df_real_test, df_fake_test], ignore_index=True, sort=False)

# Write preprocessed data
df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/train.csv', index=False)
df_valid.loc[:, ~df_valid.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/valid.csv', index=False)
df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')].sample(frac= 1).to_csv(data_path + '/test.csv', index=False)

In [10]:
train = pd.read_csv(data_path + '/train.csv')
train

,Claims,Label
0,18. A composition comprising a first material ...,0
1,10. The organic light emitting device of claim...,1
2,3. The analog-to-digital converter according t...,0
3,"13. The OFET of claim 1, wherein the hole trap...",1
4,"13. The apparatus of claim 1, wherein each of ...",1
...,...,...
12119,5. The electronic circuit as claimed in claim ...,1
12120,9. The analog-to-digital converter according t...,1
12121,"1. A method for pre-aligning a substrate, comp...",0
12122,27. The display apparatus according to claim 1...,1
